Model Architecture Overview

The proposed model is based on a ConvLSTM architecture designed for spatiotemporal sequence forecasting.
Given a sequence of historical inputs, the model simultaneously predicts multiple future time steps using a multi-output structure.

Key Parameters and Their Functions
1. filters = 16

The parameter filters specifies the number of convolutional kernels in the ConvLSTM layer, which corresponds to the number of feature maps in the hidden state.

A larger number of filters increases the model’s capacity to learn complex spatiotemporal patterns.

However, excessively large values may lead to overfitting, especially when the dataset size is limited.

In this work, filters = 16 is chosen as a balanced trade-off between representation capability and generalization performance.

2. kernel_size = (3, 3)

The kernel_size determines the spatial receptive field of the convolution operation.

A 3×3 kernel allows the model to capture local spatial dependencies while maintaining computational efficiency.

This choice is commonly adopted in spatiotemporal modeling to balance detail preservation and smoothing.

3. lookback (input sequence length)

The lookback window defines how many past time steps are used as input for prediction.

A longer lookback provides richer temporal context.

However, overly long sequences may introduce noise and increase the risk of overfitting.

The selected lookback length is sufficient to capture temporal dynamics while preserving training stability.

4. horizon = 7

The forecast horizon indicates the number of future time steps predicted by the model.

Instead of recursive forecasting, the model adopts a direct multi-step prediction strategy.

The output layer simultaneously predicts values from t+1 to t+7, reducing error accumulation across time steps.

5. Dense(horizon)

The fully connected output layer maps the shared spatiotemporal representation to multiple future targets.

Each output neuron corresponds to a specific forecast lead time.

All forecast steps are generated in parallel, rather than sequentially.

6. Loss Function (e.g., Mean Squared Error)

The loss function evaluates the discrepancy between predicted and observed values across all forecast steps.

The final loss is computed as the average error over all prediction horizons.

This encourages the model to learn both short-term and long-term predictive patterns.

7. Early Stopping and Validation Strategy

Early stopping is applied based on validation loss to prevent overfitting.

Training terminates when validation performance no longer improves.

This strategy ensures better generalization to unseen future data.

这个文件是“多步预测（multi-step）”，同时输出 t+1 到 t+7，虽然不同预测步长（t+1 与 t+7）的结果在图像中分别展示，但两者均来源于同一 ConvLSTM 模型的输出；
结果差异仅源于滑动窗口下预测起点 t0 的不同，而非模型结构或参数的差异。

改进 ： 改成小 window，filters = 8 or 16, Dropout(0.2)。在训练时，随机“关闭”一部分神经元，防止模型过度依赖某些特征

In [4]:
import os
import json
import math
import warnings
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

warnings.filterwarnings("ignore")


# ============================================================
# Configuration
# ============================================================
@dataclass
class Config:
    # CSV paths: key=region name, value=absolute path
    csv_paths: Dict[str, str] = None

    # Column names
    date_col: str = "date"
    probe_col: str = "probe_name"

    # Feature columns (8 variables)
    feature_cols: List[str] = None
    target_col: str = "sm_30cm"

    # Time-series settings
    window_list: List[int] = None
    horizon: int = 7
    split_ratio: Tuple[float, float, float] = (0.70, 0.15, 0.15)

    # Model hyperparameters (small model to reduce overfitting)
    filters: int = 16          # try 8 if still overfitting
    dropout: float = 0.2
    kernel_size: Tuple[int, int] = (1, 3)
    batch_size: int = 16
    max_epochs: int = 150
    learning_rate: float = 1e-3

    # Output root folder
    output_root: str = "outputs_multistep_context_smallmodel"

    # Reproducibility
    seed: int = 42


def set_seed(seed: int = 42):
    """Set random seeds for reproducibility."""
    np.random.seed(seed)
    tf.random.set_seed(seed)


# ============================================================
# Data utilities
# ============================================================
def load_and_clean_csv(path: str, cfg: Config) -> pd.DataFrame:
    """
    Load a CSV, parse date, keep required columns, and drop rows with missing values.
    Note: This is a simple cleaning strategy. If you need interpolation, add it here.
    """
    df = pd.read_csv(path)
    df.columns = [c.strip() for c in df.columns]

    required = [cfg.date_col, cfg.probe_col] + cfg.feature_cols
    for c in required:
        if c not in df.columns:
            raise ValueError(f"Missing column '{c}' in {path}. Available columns: {df.columns.tolist()}")

    df[cfg.date_col] = pd.to_datetime(df[cfg.date_col], errors="coerce")
    df = df.dropna(subset=[cfg.date_col]).copy()
    df = df.sort_values(cfg.date_col).reset_index(drop=True)

    # Keep only needed columns
    df = df[required].copy()

    # Convert feature columns to numeric
    for c in cfg.feature_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # Drop rows with NaN in features (simple strategy)
    df = df.dropna(subset=cfg.feature_cols).reset_index(drop=True)
    return df


def split_by_probe(df: pd.DataFrame, cfg: Config) -> Dict[str, pd.DataFrame]:
    """Split the dataframe by probe_name, each probe becomes an independent time series."""
    out = {}
    for probe_name, g in df.groupby(cfg.probe_col):
        g = g.sort_values(cfg.date_col).reset_index(drop=True)
        out[str(probe_name)] = g
    return out


def chronological_split_indices(n: int, ratios: Tuple[float, float, float]) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Chronological split indices (no shuffle):
      train: first 70%
      val  : next 15%
      test : last 15%
    """
    r_train, r_val, r_test = ratios
    assert abs((r_train + r_val + r_test) - 1.0) < 1e-9

    n_train = int(math.floor(n * r_train))
    n_val = int(math.floor(n * r_val))
    n_test = n - n_train - n_val

    idx_train = np.arange(0, n_train)
    idx_val = np.arange(n_train, n_train + n_val)
    idx_test = np.arange(n_train + n_val, n)

    return idx_train, idx_val, idx_test


# ============================================================
# Sample generation (multi-step) with context borrowing
# ============================================================
def make_multistep_samples(
    X_scaled: np.ndarray,
    y_scaled: np.ndarray,
    dates: np.ndarray,
    window: int,
    horizon: int,
    idx_split: np.ndarray,
    idx_prev: Optional[np.ndarray] = None
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Create multi-step supervised samples for a specific split.

    Context borrowing (Scheme A):
      - For validation: allow borrowing last `window` rows from train as input context.
      - For test: allow borrowing last `window` rows from (train+val) as input context.
      - Targets (y) must be strictly inside the current split => no future leakage.

    Output:
      X_seq: (num_samples, window, 1, n_features, 1)
      y_seq: (num_samples, horizon)
      d_seq: (num_samples,) forecast start date for each sample
    """
    if len(idx_split) == 0:
        return np.empty((0,)), np.empty((0,)), np.empty((0,))

    # Build context series:
    # If idx_prev is provided, prepend its tail (up to 'window' rows) before the split.
    if idx_prev is None or len(idx_prev) == 0:
        X_ctx = X_scaled[idx_split]
        y_ctx = y_scaled[idx_split]
        d_ctx = dates[idx_split]
        offset = 0
    else:
        prev_tail = idx_prev[-window:] if len(idx_prev) >= window else idx_prev
        ctx_idx = np.concatenate([prev_tail, idx_split])

        X_ctx = X_scaled[ctx_idx]
        y_ctx = y_scaled[ctx_idx]
        d_ctx = dates[ctx_idx]
        offset = len(prev_tail)

    # We define a forecast start position s in the context array:
    #   X input:  X_ctx[s-window : s]
    #   y target: y_ctx[s : s+horizon]
    # We require:
    #   - s is inside the current split region => s >= offset
    #   - full horizon target exists          => s+horizon <= len(X_ctx)
    starts = []
    for s in range(offset, len(X_ctx) - horizon + 1):
        if s - window < 0:
            continue
        starts.append(s)

    if len(starts) == 0:
        return np.empty((0,)), np.empty((0,)), np.empty((0,))

    n_features = X_ctx.shape[1]
    X_seq = np.zeros((len(starts), window, 1, n_features, 1), dtype=np.float32)
    y_seq = np.zeros((len(starts), horizon), dtype=np.float32)
    d_seq = np.zeros((len(starts),), dtype="datetime64[ns]")

    for i, s in enumerate(starts):
        Xw = X_ctx[s - window: s]                # shape (window, n_features)
        yh = y_ctx[s: s + horizon].reshape(-1)   # shape (horizon,)

        X_seq[i, :, 0, :, 0] = Xw
        y_seq[i, :] = yh
        d_seq[i] = d_ctx[s]                      # forecast start date

    return X_seq, y_seq, d_seq


# ============================================================
# Model: ConvLSTM (small capacity)
# ============================================================
def build_convlstm_model(window: int, n_features: int, horizon: int, cfg: Config) -> tf.keras.Model:
    """
    ConvLSTM-style model for daily tabular time series.
    We treat the feature dimension as a "spatial axis": (1 x n_features), channel=1.
    """
    tf.keras.backend.clear_session()

    inp = layers.Input(shape=(window, 1, n_features, 1))

    x = layers.ConvLSTM2D(
        filters=cfg.filters,
        kernel_size=cfg.kernel_size,
        padding="same",
        activation="tanh",
        recurrent_activation="sigmoid",
        return_sequences=False
    )(inp)

    x = layers.BatchNormalization()(x)
    x = layers.Dropout(cfg.dropout)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(32, activation="relu")(x)
    x = layers.Dropout(cfg.dropout)(x)

    out = layers.Dense(horizon, activation="linear")(x)

    model = models.Model(inputs=inp, outputs=out)
    opt = tf.keras.optimizers.Adam(learning_rate=cfg.learning_rate)
    model.compile(optimizer=opt, loss="mse")
    return model


# ============================================================
# Plotting & metrics
# ============================================================
def plot_loss(history: tf.keras.callbacks.History, out_png: str, title: str):
    """Plot train and validation loss curves."""
    plt.figure(figsize=(8, 5))
    plt.plot(history.history.get("loss", []), label="train_loss")
    plt.plot(history.history.get("val_loss", []), label="val_loss")
    plt.title(title)
    plt.xlabel("epoch")
    plt.ylabel("loss (MSE on scaled y)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_png, dpi=150)
    plt.close()


def plot_true_vs_pred(dates: np.ndarray, y_true: np.ndarray, y_pred: np.ndarray, out_png: str, title: str, ylabel: str):
    """Plot actual vs predicted series."""
    plt.figure(figsize=(8, 5))
    plt.plot(dates, y_true, label="actual")
    plt.plot(dates, y_pred, label="prediction")
    plt.title(title)
    plt.xlabel("date")
    plt.ylabel(ylabel)
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_png, dpi=150)
    plt.close()


def rmse(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))


def evaluate_multistep(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    """
    Evaluate multi-step predictions.
    y_true, y_pred: shape (N, H) in original scale.
    """
    H = y_true.shape[1]
    metrics = {}

    metrics["MAE_t+1"] = float(mean_absolute_error(y_true[:, 0], y_pred[:, 0]))
    metrics["RMSE_t+1"] = rmse(y_true[:, 0], y_pred[:, 0])

    metrics[f"MAE_t+{H}"] = float(mean_absolute_error(y_true[:, H - 1], y_pred[:, H - 1]))
    metrics[f"RMSE_t+{H}"] = rmse(y_true[:, H - 1], y_pred[:, H - 1])

    mae_steps = []
    rmse_steps = []
    for k in range(H):
        mae_steps.append(mean_absolute_error(y_true[:, k], y_pred[:, k]))
        rmse_steps.append(rmse(y_true[:, k], y_pred[:, k]))

    metrics["MAE_avg"] = float(np.mean(mae_steps))
    metrics["RMSE_avg"] = float(np.mean(rmse_steps))
    return metrics


# ============================================================
# Training pipeline per (region, probe, window)
# ============================================================
def run_one_experiment(df_probe: pd.DataFrame, region: str, probe: str, window: int, cfg: Config):
    """
    Train/evaluate/save artifacts for one probe and one window size.

    Key points:
      - Scalers are fitted on TRAIN only (no leakage).
      - Validation borrows input context from TRAIN tail.
      - Test borrows input context from (TRAIN+VAL) tail.
      - Targets remain inside VAL/TEST splits.
    """
    out_dir = os.path.join(cfg.output_root, f"window_{window}", region, probe)
    os.makedirs(out_dir, exist_ok=True)

    # Prepare arrays
    dates = df_probe[cfg.date_col].values
    X = df_probe[cfg.feature_cols].values.astype(np.float32)  # (N, F)
    y = df_probe[[cfg.target_col]].values.astype(np.float32)  # (N, 1)

    N, F = X.shape

    # Basic sanity check: ensure we have enough overall rows
    if N < (window + cfg.horizon + 5):
        print(f"[SKIP] {region}/{probe} window={window} horizon={cfg.horizon} -> too few rows: {N}")
        return

    # Split indices (chronological)
    idx_train, idx_val, idx_test = chronological_split_indices(N, cfg.split_ratio)

    # Fit scalers on TRAIN only
    scaler_x = StandardScaler()
    scaler_y = StandardScaler()
    scaler_x.fit(X[idx_train])
    scaler_y.fit(y[idx_train])

    Xs = scaler_x.transform(X)
    ys = scaler_y.transform(y)

    # Generate samples:
    # Train samples use only train indices (no borrowing)
    X_train, y_train, d_train = make_multistep_samples(
        Xs, ys, dates, window, cfg.horizon, idx_train, idx_prev=None
    )

    # Validation borrows from train tail
    X_val, y_val, d_val = make_multistep_samples(
        Xs, ys, dates, window, cfg.horizon, idx_val, idx_prev=idx_train
    )

    # Test borrows from (train + val) tail (recommended)
    idx_prev_test = np.concatenate([idx_train, idx_val])
    X_test, y_test, d_test = make_multistep_samples(
        Xs, ys, dates, window, cfg.horizon, idx_test, idx_prev=idx_prev_test
    )

    # If any split cannot form samples, skip
    if len(X_train) == 0 or len(X_val) == 0 or len(X_test) == 0:
        print(f"[SKIP] {region}/{probe} window={window} -> empty split samples "
              f"(train={len(X_train)}, val={len(X_val)}, test={len(X_test)})")
        return

    # Debug info (helps you understand sample counts)
    print(f"[INFO] {region}/{probe} window={window} | rows={N} | "
          f"samples train={len(X_train)}, val={len(X_val)}, test={len(X_test)}")

    # Build a small ConvLSTM model
    model = build_convlstm_model(window=window, n_features=F, horizon=cfg.horizon, cfg=cfg)

    # Callbacks: early stopping prevents unnecessary training
    early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6, verbose=1)

    # Train
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=cfg.max_epochs,
        batch_size=cfg.batch_size,
        callbacks=[early_stop, reduce_lr],
        verbose=1
    )

    # Save history
    hist_df = pd.DataFrame({
        "epoch": np.arange(len(history.history["loss"])),
        "loss": history.history["loss"],
        "val_loss": history.history["val_loss"]
    })
    hist_csv = os.path.join(out_dir, "history.csv")
    hist_df.to_csv(hist_csv, index=False)

    # Plot loss curve
    loss_png = os.path.join(out_dir, "loss_train_val.png")
    plot_loss(history, loss_png, title=f"LOSS: {region}/{probe} (window={window}, horizon={cfg.horizon})")

    # Predict on test
    y_pred_scaled = model.predict(X_test, verbose=0)  # (Ntest, H)

    # Inverse scaling safely: flatten -> inverse -> reshape
    y_true = scaler_y.inverse_transform(y_test.reshape(-1, 1)).reshape(-1, cfg.horizon)
    y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).reshape(-1, cfg.horizon)

    # Align x-axis to TARGET dates (more intuitive for step plots)
    d_test_dt = pd.to_datetime(d_test)

    # Step 1 plot (t+1)
    step1_dates = d_test_dt + pd.to_timedelta(1, unit="D")
    test_step1_png = os.path.join(out_dir, "test_true_vs_pred_step1.png")
    plot_true_vs_pred(
        dates=step1_dates,
        y_true=y_true[:, 0],
        y_pred=y_pred[:, 0],
        out_png=test_step1_png,
        title=f"Test True vs Pred (t+1): {region}/{probe} | window={window}",
        ylabel="sm_30cm (m3/m3)"
    )

    # Step H plot (t+H)
    stepH_dates = d_test_dt + pd.to_timedelta(cfg.horizon, unit="D")
    test_stepH_png = os.path.join(out_dir, f"test_true_vs_pred_step{cfg.horizon}.png")
    plot_true_vs_pred(
        dates=stepH_dates,
        y_true=y_true[:, cfg.horizon - 1],
        y_pred=y_pred[:, cfg.horizon - 1],
        out_png=test_stepH_png,
        title=f"Test True vs Pred (t+{cfg.horizon}): {region}/{probe} | window={window}",
        ylabel="sm_30cm (m3/m3)"
    )

    # Save test comparison CSV (long format)
    rows = []
    for i in range(len(d_test_dt)):
        start_date = d_test_dt[i]
        for k in range(cfg.horizon):
            target_date = start_date + pd.to_timedelta(k + 1, unit="D")
            rows.append({
                "forecast_start_date": start_date,
                "target_date": target_date,
                "step": k + 1,
                "y_true": float(y_true[i, k]),
                "y_pred": float(y_pred[i, k]),
            })
    test_csv = os.path.join(out_dir, "test_compare_multistep.csv")
    pd.DataFrame(rows).to_csv(test_csv, index=False)

    # Compute and save summary metrics
    metrics = evaluate_multistep(y_true, y_pred)
    metrics.update({
        "region": region,
        "probe_name": probe,
        "window": window,
        "horizon": cfg.horizon,
        "n_rows": int(N),
        "n_train_samples": int(len(X_train)),
        "n_val_samples": int(len(X_val)),
        "n_test_samples": int(len(X_test)),
        "filters": cfg.filters,
        "dropout": cfg.dropout,
    })
    metrics_path = os.path.join(out_dir, "summary_metrics.json")
    with open(metrics_path, "w", encoding="utf-8") as f:
        json.dump(metrics, f, indent=2)

    # Save model and scalers (for reuse)
    model_path = os.path.join(out_dir, "model.h5")
    model.save(model_path)

    joblib.dump(scaler_x, os.path.join(out_dir, "scaler_x.pkl"))
    joblib.dump(scaler_y, os.path.join(out_dir, "scaler_y.pkl"))

    # Forecast next H days from the LAST available day in this probe
    last_X_window = Xs[-window:]  # scaled features for the last window
    last_X_seq = last_X_window.reshape(1, window, 1, F, 1).astype(np.float32)

    future_pred_scaled = model.predict(last_X_seq, verbose=0).reshape(-1, 1)  # (H,1)
    future_pred = scaler_y.inverse_transform(future_pred_scaled).reshape(-1)  # (H,)

    last_date = pd.to_datetime(df_probe[cfg.date_col].iloc[-1])
    future_dates = [last_date + pd.to_timedelta(i + 1, unit="D") for i in range(cfg.horizon)]

    future_df = pd.DataFrame({"forecast_date": future_dates, "pred_sm_30cm": future_pred})
    future_csv = os.path.join(out_dir, f"last_date_forecast_next_{cfg.horizon}_days.csv")
    future_df.to_csv(future_csv, index=False)

    # Forecast plot
    future_png = os.path.join(out_dir, f"last_date_forecast_next_{cfg.horizon}_days.png")
    plt.figure(figsize=(8, 5))
    plt.plot(future_df["forecast_date"], future_df["pred_sm_30cm"], marker="o")
    plt.title(f"Forecast next {cfg.horizon} days from last date: {region}/{probe} | window={window}")
    plt.xlabel("date")
    plt.ylabel("pred sm_30cm (m3/m3)")
    plt.tight_layout()
    plt.savefig(future_png, dpi=150)
    plt.close()

    print(f"[OK] Saved outputs -> {out_dir}")


# ============================================================
# Main runner
# ============================================================
def main():
    cfg = Config()

    # --- Update these paths if your local paths differ ---
    cfg.csv_paths = {
        "Grandvillers_Sec": r"D:\UV Projet\Soil Moisture\Grandvillers_Sec.csv",
        "Grandvillers_Canon": r"D:\UV Projet\Soil Moisture\Grandvillers-Canon.csv",
        "Grandvillers_Robot_20": r"D:\UV Projet\Soil Moisture\Grandvillers-Robot-20.csv",
        "Grandvillers_Robot": r"D:\UV Projet\Soil Moisture\Grandvillers-Robot.csv",
    }

    # 8 input variables (including sm_30cm as an autoregressive feature)
    cfg.feature_cols = [
        "sm_30cm",
        "irrig_mm",
        "IRRAD",
        "TMIN",
        "TMAX",
        "VAP",
        "WIND",
        "RAIN",
    ]

    # Window sizes to compare
    cfg.window_list = [14, 30]  # you can set [14] only if you want

    # Small model configuration
    cfg.filters = 16   # try 8 if you want an even smaller model
    cfg.dropout = 0.2

    set_seed(cfg.seed)
    os.makedirs(cfg.output_root, exist_ok=True)

    all_probes = set()

    for region, path in cfg.csv_paths.items():
        if not os.path.exists(path):
            print(f"[SKIP] File not found: {path}")
            continue

        df = load_and_clean_csv(path, cfg)
        probe_map = split_by_probe(df, cfg)

        print(f"\nRegion={region} | probes={list(probe_map.keys())} (count={len(probe_map)})")
        for p in probe_map.keys():
            all_probes.add(p)

        for probe_name, df_probe in probe_map.items():
            for window in cfg.window_list:
                run_one_experiment(df_probe, region, probe_name, window, cfg)

    print("\nAll unique probe_name values across 4 CSV:")
    print(sorted(list(all_probes)))
    print(f"Total unique probe_name: {len(all_probes)}")
    print(f"\nDone. Outputs saved under: {cfg.output_root}")


if __name__ == "__main__":
    main()



Region=Grandvillers_Sec | probes=['Sec'] (count=1)
[INFO] Grandvillers_Sec/Sec window=14 | rows=115 | samples train=60, val=11, test=12
Epoch 1/150
4/4 [==============================] - 0s 27ms/step - loss: 1.8525 - val_loss: 0.7692 - lr: 0.0010
Epoch 2/150
4/4 [==============================] - 0s 8ms/step - loss: 1.1920 - val_loss: 0.7716 - lr: 0.0010
Epoch 3/150
4/4 [==============================] - 0s 8ms/step - loss: 0.9257 - val_loss: 0.7715 - lr: 0.0010
Epoch 4/150
4/4 [==============================] - 0s 7ms/step - loss: 0.8256 - val_loss: 0.7713 - lr: 0.0010
Epoch 5/150
4/4 [==============================] - 0s 8ms/step - loss: 0.6853 - val_loss: 0.7738 - lr: 0.0010
Epoch 6/150
1/4 [======>.......................] - ETA: 0s - loss: 0.5623
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 8ms/step - loss: 0.6184 - val_loss: 0.7788 - lr: 0.0010
Epoch 7/150
4/4 [==============================] - 0s 8ms/st

# The ConvLSTM model successfully captured the smooth temporal dynamics of soil moisture under normal conditions. However, abrupt drops observed at the end of the test period were present in the original measurements and were not predictable from historical meteorological and soil moisture inputs. This indicates that the model learned stable hydrological patterns rather than sensor anomalies or abrupt external disturbances.